<a href="https://colab.research.google.com/github/rswijesena/AI/blob/main/Advance_OpenAI_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openai
!pip install panda
!pip install pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.3/320.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for panda: filename=panda-0.3.1-py3-none-any.whl size=7239 sha256=5ed1e479f92fb768f8b3fb44130f28ac41cd4d3dbd7bf47450e57e7aa680cc0c
  Stored in directory: /root/.cache/pip/wheels/0e/8b/c3/ff9cbde1fffd8071cff8367a86f0350a1ce30a8d31b6a432e9
Successfully built panda
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import openai
from openai import OpenAI

openAIKey=""

client = OpenAI(api_key=openAIKey)

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {
            "role": "user",
            "content": "When's the next flight from delhi to mumbai?"
        }
    ],
)
function_description = [
    {
        "name": "get_flight_info",
        "description": "Get the next flight from delhi to mumbai",
        "parameters": {
            "type": "object",
            "properties": {
                "from": {"type": "string", "description": "The depature airport e.g DEL"},
                "to": {"type": "string", "description": "The destination airport e.g SYD"},
                "date": {"type": "string"},
            },
            "required": ["from", "to"],
        },
    }
]

In [24]:
user_prompt = "When's the next flight from Colombo to Sydney? "

In [25]:
respones2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": user_prompt
        }
    ],
    #Add function calling
    functions=function_description,
    function_call="auto",
)

In [26]:
import json
from datetime import datetime, timedelta
def get_flight_info(from_airport, to_airport):
  """Get Flight information between two airports"""
  #Example out put

  flight_info = {
      "from_airport": from_airport,
      "to_airport": to_airport,
      "date": str(datetime.now() + timedelta(hours=2)),
      "airline" : "Qantas",
      "flight_number": "QF466"
  }
  return json.dumps(flight_info)

In [31]:
origin = json.loads(respones2.choices[0].message.function_call.arguments).get("from")
destination = json.loads(respones2.choices[0].message.function_call.arguments).get("to")

origin,destination

('CMB', 'SYD')

In [29]:
response3 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": user_prompt
        },
        {
            "role": "function",
            "name": "get_flight_info",
            "content": get_flight_info(origin, destination)
        }
    ],
    #Add function calling
    functions=function_description,
    function_call="auto" # specify the function call

)

response3.choices[0].message.content

'The next flight from Colombo (CMB) to Sydney (SYD) is on May 14, 2024. The flight is operated by Qantas with flight number QF466.'